In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
data = pd.read_csv('titanic.csv')

In [2]:
data.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Gender   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [3]:
def checkAdult(age):
    if age>=18:
        return "Adult"
    else:
        return "Child"
    
data["Adult/Child"]=data["Age"].apply(checkAdult)
data.head() 

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Gender   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked Adult/Child  
0      0         A/5 21171   7.2500   NaN        S       Adult  
1      0          PC 17599  71.2833   C85        C       Adult  
2      0  STON/O2. 3101282   7.9250   NaN        S       Adult  
3      0            113803  53.1000  C123        S       Adult  
4      0            373450   8.0500   NaN        S       Adult

In [4]:
data.groupby(['Gender'])['Survived'].sum()

Gender
female    233
male      109
Name: Survived, dtype: int64

In [5]:
trainingData=data[["Pclass","Adult/Child","Gender","Survived"]]
trainingData.head()

Pclass Adult/Child  Gender  Survived
0       3       Adult    male         0
1       1       Adult  female         1
2       3       Adult  female         1
3       1       Adult  female         1
4       3       Adult    male         0

In [6]:
def catToNum(series):
    series = series.astype('category')
    return series.cat.codes

catData=trainingData[["Pclass","Adult/Child","Gender","Survived"]].apply(catToNum)
trainingData[["Pclass","Adult/Child","Gender","Survived"]]=catData
trainingData.head()

C:\Users\Faizy\AppData\Local\Temp\ipykernel_12356\2812049076.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingData[["Pclass","Adult/Child","Gender","Survived"]]=catData


Pclass  Adult/Child  Gender  Survived
0       2            0       1         0
1       0            0       0         1
2       2            0       0         1
3       0            0       0         1
4       2            0       1         0

In [7]:
len(trainingData)

891

In [8]:
trainingData = trainingData.dropna()
len(trainingData)

891

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(trainingData, test_size = 0.2)

In [10]:
train.head()

Pclass  Adult/Child  Gender  Survived
127       2            0       1         1
140       2            1       0         0
501       2            0       0         0
420       2            1       1         0
294       2            0       1         0

In [11]:
test.head()

Pclass  Adult/Child  Gender  Survived
183       1            1       1         1
209       0            0       1         1
619       1            0       1         0
0         2            0       1         0
819       2            1       1         0

In [12]:
len(train)

712

In [13]:
len(test)

179

In [14]:
def cal_entropy(col):
    # first convert the column to numpy array and then find the unique elements and their frequency
    col = np.array(col)
    elements,freq = np.unique(col,return_counts = True)
    # print(elements,freq)
    entropy = np.sum([(-freq[i]/np.sum(freq))*np.log2(freq[i]/np.sum(freq)) for i in range(len(elements))])
    return entropy


In [15]:
entropy = cal_entropy(train["Survived"])
print(entropy)

0.9659855405492415


In [16]:
def InfoGain(data,split_attribute_name,target_name="Survived"):
    """
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset

    parent_entropy = cal_entropy(data[target_name])
    print(f"Total dataset/Parent Entropy: {parent_entropy}")

    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    print(f"Split attribute Name: {split_attribute_name}")
    print(f"Split attribute values: {vals}")
    print(f"Split attribute values counts: {counts}") 

    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*cal_entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    print(f"Weighted Entropy: {Weighted_Entropy}")

    #Calculate the information gain
    print(f"Calculating Information Gain using: {parent_entropy} - {Weighted_Entropy}")
    Information_Gain = parent_entropy - Weighted_Entropy
    print(f"Information Gain: {Information_Gain}")
    return Information_Gain
       

In [17]:
#Test the function 
InfoGain(data=train,split_attribute_name="Pclass",target_name="Survived") 

Total dataset/Parent Entropy: 0.9659855405492415
Split attribute Name: Pclass
Split attribute values: [0 1 2]
Split attribute values counts: [173 155 384]
Weighted Entropy: 0.8860320861153153
Calculating Information Gain using: 0.9659855405492415 - 0.8860320861153153
Information Gain: 0.07995345443392621


0.07995345443392621

In [18]:
def ID3(data, originaldata, features, target_attribute_name="Survived", parent_node_class=None):
    # Define the stopping criteria
    # If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1: #checks only 1 uniquie attribute of current dataset
        return np.unique(data[target_attribute_name])[0]

    # If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data) == 0: #return the most common value of target attribute
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name], return_counts=True)[1])]

    # If the feature space is empty, return the mode target feature value of the direct parent node
    elif len(features) == 0: #if no features to split on then return the most common value of target attribute in current dataset
        return parent_node_class

    # If none of the above holds true, grow the tree!
    else:
        # Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]

        #This calculates the information gain for each feature in the current dataset, then selects the feature with the highest information gain as the "best_feature".
        item_values = [InfoGain(data, feature, target_attribute_name) for feature in features]
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        # Create the tree structure by, make tree node, the best feature and then the sub trees 
        tree = {best_feature: {}}

        # Remove the feature with the best information gain from the feature space
        features = [i for i in features if i != best_feature]

        # Grow a branch under the root node for each possible value of the root node feature
        for value in np.unique(data[best_feature]):
            value = value
            # Split the dataset along the value of the feature with the largest information gain and create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()

            # Call the ID3 algorithm for each of those sub_datasets with the new parameters
            subtree = ID3(sub_data, data, features, target_attribute_name, parent_node_class)

            # Add the subtree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree

        return (tree)

        

In [19]:
tree = ID3(train,train,train.columns[:-1])
pprint(tree)

Total dataset/Parent Entropy: 0.9659855405492415
Split attribute Name: Pclass
Split attribute values: [0 1 2]
Split attribute values counts: [173 155 384]
Weighted Entropy: 0.8860320861153153
Calculating Information Gain using: 0.9659855405492415 - 0.8860320861153153
Information Gain: 0.07995345443392621
Total dataset/Parent Entropy: 0.9659855405492415
Split attribute Name: Adult/Child
Split attribute values: [0 1]
Split attribute values counts: [483 229]
Weighted Entropy: 0.9656943453158204
Calculating Information Gain using: 0.9659855405492415 - 0.9656943453158204
Information Gain: 0.0002911952334210266
Total dataset/Parent Entropy: 0.9659855405492415
Split attribute Name: Gender
Split attribute values: [0 1]
Split attribute values counts: [254 458]
Weighted Entropy: 0.7525645236871441
Calculating Information Gain using: 0.9659855405492415 - 0.7525645236871441
Information Gain: 0.21342101686209736
Total dataset/Parent Entropy: 0.8205046630531412
Split attribute Name: Pclass
Split att

Weighted Entropy: 0.8169197015093461
Calculating Information Gain using: 0.8205046630531412 - 0.8169197015093461
Information Gain: 0.003584961543795062
Total dataset/Parent Entropy: 0.10559103637519526
Split attribute Name: Adult/Child
Split attribute values: [0. 1.]
Split attribute values counts: [58 14]
Weighted Entropy: 0.10122454041679042
Calculating Information Gain using: 0.10559103637519526 - 0.10122454041679042
Information Gain: 0.00436649595840484
Total dataset/Parent Entropy: 0.4262286569981449
Split attribute Name: Adult/Child
Split attribute values: [0. 1.]
Split attribute values counts: [57 12]
Weighted Entropy: 0.40103280235531974
Calculating Information Gain using: 0.4262286569981449 - 0.40103280235531974
Information Gain: 0.025195854642825144
Total dataset/Parent Entropy: 0.9994915114306261
Split attribute Name: Adult/Child
Split attribute values: [0. 1.]
Split attribute values counts: [52 61]
Weighted Entropy: 0.9736543608157822
Calculating Information Gain using: 0.99